In [72]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import cv2
from albumentations import Rotate
from scipy import ndimage
import imutils
from skimage import data
from skimage import color
from skimage.filters import meijering, sato, frangi, hessian
from skimage.morphology import medial_axis, skeletonize, thin, remove_small_objects
import rdp
import math
from tqdm import tqdm
import sys
sys.path.append('../Scripts/')
from segmentation_utils import *
from metrics import *
from image_utils import *
gt = True

In [69]:
def calculate(path_list):
    df = pd.DataFrame(columns = ['img_id','tort', 'dm', 'ip', 'tp', 'sd'])
    all_metrics = []
    all_img_ids = []
    for i, path in tqdm(enumerate(path_list)):
        img_orig = cv2.imread(path, 0)
        seg_img = segment(img_orig)
        all_points = get_points(seg_img)
        all_unique = get_all_unique(all_points)
        points = get_turning_points(seg_img,tolerance=2,min_angle=np.pi*0.15)
        vessel_metrics = []
        for j in range(len(all_unique)):
            tort,dm,ip,tp,sd = tortousity(all_unique[j])
            vessel_metrics.append([tort,dm,ip,tp,sd])
            all_img_ids.append(path.split('/')[-1])
        all_metrics.append(np.array(vessel_metrics))
    df.img_id = all_img_ids
    df.iloc[:, 1:6] = np.concatenate(all_metrics)
    return df

def calculate_gt(path_list):
    df = pd.DataFrame(columns = ['img_id','tort', 'dm', 'ip', 'tp', 'sd', 'Class'])
    all_metrics = []
    all_img_ids = []
    for i, path in tqdm(enumerate(path_list)):
        img_orig = cv2.imread(path, 0)
        img_orig = cv2.resize(img_orig,(210, 210))
        seg_img = skeletonize(img_orig<240)
        all_points = get_points(seg_img)
        all_unique = get_all_unique(all_points)
        points = get_turning_points(seg_img,tolerance=2,min_angle=np.pi*0.15)
        vessel_metrics = []
        for j in range(len(all_unique)):
            tort,dm,ip,tp,sd = tortousity(all_unique[j])
            cls = path.split('/')[-1][0]
            vessel_metrics.append([tort,dm,ip,tp,sd, cls])
            all_img_ids.append(path.split('/')[-1])
        all_metrics.append(np.array(vessel_metrics))
    df.img_id = all_img_ids
    df.iloc[:, 1:7] = np.concatenate(all_metrics)
    return df

In [70]:
img_paths1 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/*')) #Original Folder
img_paths2 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/*')) #Original Folder
img_paths3 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/*')) #Original Folder
if gt==False:
    img_paths = img_paths1 + img_paths2 + img_paths3
    df = calculate(img_paths)
else:
    img_paths = sorted(glob.glob('../Data/SPIE_2020_RBV_manualmarking/*'))
    df = calculate_gt(img_paths)

13it [12:08, 51.26s/it]../Scripts/metrics.py:287: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
16it [14:05, 42.62s/it]../Scripts/metrics.py:292: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
59it [49:32, 50.39s/it]


In [74]:
if gt==False:
    img_paths1 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/')) #Original Folder
    img_paths2 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/')) #Original Folder
    img_paths3 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/')) #Original Folder
    class_mapping1 = { i : 'Emmetropia' for i in img_paths1 }
    class_mapping2 = { i : 'Myopia' for i in img_paths2 }
    class_mapping3 = { i : 'Hypermetropia' for i in img_paths3 }
    class_mapping = {**class_mapping1, **class_mapping2, **class_mapping3}
    class_mapping = {**class_mapping1, **class_mapping2, **class_mapping3}
    df['Class'] = ''
    for i in range(len(df)):
        df.Class.iloc[i] = class_mapping[df.img_id.iloc[i]]

In [75]:
df.Class.value_counts()

H    643
E    588
M    501
Name: Class, dtype: int64

In [76]:
if gt:
    df.to_csv('../Data/complete_data_gt.csv', index = False)
else:
    df.to_csv('../Data/complete_data.csv', index = False)

In [77]:
df = pd.read_csv('../Data/complete_data_gt.csv')

In [78]:
df.groupby(['Class', 'img_id']).tp.describe()

count      mean  \
Class img_id                                                                
E     E_1193671_Jandyal_sourabh__1193671_19951205_Mal...   30.0  1.066667   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   35.0  1.485714   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   27.0  1.222222   
      E_1204144_Divya_A__1204144_19940912_Female_OD.jpg    28.0  1.678571   
      E_1204144_Divya_A__1204144_19940912_Female_OS.jpg    24.0  1.833333   
      E_1204147_Malika_M__1204147_19961230_Female_OD.jpg   34.0  1.088235   
      E_1204147_Malika_M__1204147_19961230_Female_OS.jpg   27.0  1.111111   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   28.0  1.321429   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   33.0  1.303030   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   35.0  1.085714   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   33.0  1.363636   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   30.0  1.933333   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   29.0  1.517241   
      E_209398_SUMIT_CHOUDHARY__209398_19850606_Male_...   28.0  1.285714   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   25.0  1.920000   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   24.0  1.916667   
      E_301279_Vineet_Ratra__301279_19700517_Male_OD.jpg   31.0  1.225806   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   28.0  1.678571   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   25.0  1.560000   
      E_787513_SUSANTA_SINGHA__787513_19801019_Male_O...   34.0  1.294118   
H     H_1189801_KOWSIGAN_M__1189801_19950306_Male_OD.jpg   25.0  1.640000   
      H_1189801_KOWSIGAN_M__1189801_19950306_Male_OS.jpg   27.0  1.518519   
      H_2144798_BISWA NATH ADAK_Male_OS.jpg                29.0  1.137931   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   39.0  2.256410   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   35.0  1.771429   
      H_2836849__BALASUBRAMANYAN_S__2836849_19380815_...   36.0  1.805556   
      H_2850656_ILANGOVAN_V__2850656_19471005_Male_OS...   15.0  2.000000   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   34.0  1.970588   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   30.0  1.766667   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OD.jpg    37.0  1.702703   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OS.jpg    27.0  1.962963   
      H_3489266_SENTHIL KUMAR_R__3489266_19751119_Mal...   29.0  2.034483   
      H_4275323_MEENAKSHI_SUNDARAM__4275323_19520922_...   32.0  1.812500   
      H_429419_SATHYAMOORTHY_NP__429419_19510101_Fema...   36.0  1.861111   
      H_4297748_DILIP_JAIN__4297748_19711011_Male_OS.jpg   46.0  1.673913   
      H_4710617_ANBU HEPZIBAH_MUTHABARANAM__4710617_1...   26.0  1.884615   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   37.0  1.297297   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   33.0  1.333333   
      H_519266_SIB SANKAR_PATRA__519266_19511006_Male...   31.0  2.258065   
      H_902196_VISWANATHAN S___902196_19710803_Male_O...   39.0  1.487179   
M     M_ABHIJIT_ROY__2886965_19990928_Male_OD.jpg          40.0  2.050000   
      M_ABHIJIT_ROY__2886965_19990928_Male_OS.jpg          29.0  2.482759   
      M_ABIHA TAHSIN_CHOWDHURY__2889149_19881204_Fema...   32.0  2.093750   
      M_AMOSE_T__1178927_19920112_Male_OD.jpg              23.0  2.000000   
      M_AMOSE_T__1178927_19920112_Male_OS.jpg              25.0  1.560000   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD.jpg        22.0  1.681818   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD_1.jpg      28.0  1.892857   
      M_Aishwarya_R__1184391_19961122_Female_OD.jpg        20.0  1.550000   
      M_Aishwarya_R__1184391_19961122_Female_OS.jpg        22.0  1.863636   
      M_Akbar_Rukshar__1178658_19940804_Female_OD.jpg      25.0  1.360000   
      M_Akbar_Rukshar__1178658_19940804_Female_OS.jpg 

In [79]:
df.groupby(['Class', 'img_id']).sd.describe()

count       mean  \
Class img_id                                                                 
E     E_1193671_Jandyal_sourabh__1193671_19951205_Mal...   30.0  23.746824   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   35.0  29.731405   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   27.0  26.953989   
      E_1204144_Divya_A__1204144_19940912_Female_OD.jpg    28.0  31.560831   
      E_1204144_Divya_A__1204144_19940912_Female_OS.jpg    24.0  32.507661   
      E_1204147_Malika_M__1204147_19961230_Female_OD.jpg   34.0  21.656409   
      E_1204147_Malika_M__1204147_19961230_Female_OS.jpg   27.0  27.118776   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   28.0  23.208709   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   33.0  26.118917   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   35.0  29.491360   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   33.0  27.736151   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   30.0  33.647457   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   29.0  18.813460   
      E_209398_SUMIT_CHOUDHARY__209398_19850606_Male_...   28.0  36.093564   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   25.0  30.637454   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   24.0  29.370228   
      E_301279_Vineet_Ratra__301279_19700517_Male_OD.jpg   31.0  29.971982   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   28.0  33.801491   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   25.0  28.885572   
      E_787513_SUSANTA_SINGHA__787513_19801019_Male_O...   34.0  27.934953   
H     H_1189801_KOWSIGAN_M__1189801_19950306_Male_OD.jpg   25.0  34.609501   
      H_1189801_KOWSIGAN_M__1189801_19950306_Male_OS.jpg   27.0  29.306714   
      H_2144798_BISWA NATH ADAK_Male_OS.jpg                29.0  28.280747   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   39.0  34.919588   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   35.0  30.142933   
      H_2836849__BALASUBRAMANYAN_S__2836849_19380815_...   36.0  36.875231   
      H_2850656_ILANGOVAN_V__2850656_19471005_Male_OS...   15.0  28.885211   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   34.0  25.875821   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   30.0  30.123670   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OD.jpg    37.0  29.535854   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OS.jpg    27.0  44.467999   
      H_3489266_SENTHIL KUMAR_R__3489266_19751119_Mal...   29.0  33.512079   
      H_4275323_MEENAKSHI_SUNDARAM__4275323_19520922_...   32.0  27.962533   
      H_429419_SATHYAMOORTHY_NP__429419_19510101_Fema...   36.0  29.638361   
      H_4297748_DILIP_JAIN__4297748_19711011_Male_OS.jpg   46.0  24.151044   
      H_4710617_ANBU HEPZIBAH_MUTHABARANAM__4710617_1...   26.0  28.064880   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   37.0  24.132197   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   33.0  31.986954   
      H_519266_SIB SANKAR_PATRA__519266_19511006_Male...   31.0  30.473432   
      H_902196_VISWANATHAN S___902196_19710803_Male_O...   39.0  29.359240   
M     M_ABHIJIT_ROY__2886965_19990928_Male_OD.jpg          40.0  29.162410   
      M_ABHIJIT_ROY__2886965_19990928_Male_OS.jpg          29.0  32.359073   
      M_ABIHA TAHSIN_CHOWDHURY__2889149_19881204_Fema...   32.0  32.336553   
      M_AMOSE_T__1178927_19920112_Male_OD.jpg              23.0  38.743502   
      M_AMOSE_T__1178927_19920112_Male_OS.jpg              25.0  32.949551   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD.jpg        22.0  33.545022   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD_1.jpg      28.0  36.939743   
      M_Aishwarya_R__1184391_19961122_Female_OD.jpg        20.0  39.889787   
      M_Aishwarya_R__1184391_19961122_Female_OS.jpg        22.0  36.960331   
      M_Akbar_Rukshar__1178658_19940804_Female_OD.jpg      25.0  24.448378   
  

In [80]:
df.groupby(['Class', 'img_id']).dm.describe()

count      mean  \
Class img_id                                                                
E     E_1193671_Jandyal_sourabh__1193671_19951205_Mal...   30.0  0.455825   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   35.0  0.397598   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   27.0  0.317853   
      E_1204144_Divya_A__1204144_19940912_Female_OD.jpg    28.0  0.406551   
      E_1204144_Divya_A__1204144_19940912_Female_OS.jpg    24.0  0.334734   
      E_1204147_Malika_M__1204147_19961230_Female_OD.jpg   34.0  0.584421   
      E_1204147_Malika_M__1204147_19961230_Female_OS.jpg   27.0  0.313670   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   28.0  0.316723   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   33.0  0.311458   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   35.0  0.502745   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   33.0  0.266059   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   30.0  0.634671   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   29.0  0.428982   
      E_209398_SUMIT_CHOUDHARY__209398_19850606_Male_...   27.0  0.493360   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   25.0  0.352727   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   24.0  0.259574   
      E_301279_Vineet_Ratra__301279_19700517_Male_OD.jpg   30.0  0.509953   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   28.0  0.435769   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   25.0  0.206035   
      E_787513_SUSANTA_SINGHA__787513_19801019_Male_O...   34.0  0.435092   
H     H_1189801_KOWSIGAN_M__1189801_19950306_Male_OD.jpg   25.0  0.465355   
      H_1189801_KOWSIGAN_M__1189801_19950306_Male_OS.jpg   27.0  0.517031   
      H_2144798_BISWA NATH ADAK_Male_OS.jpg                28.0  0.426809   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   39.0  0.717748   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   35.0  0.355749   
      H_2836849__BALASUBRAMANYAN_S__2836849_19380815_...   36.0  0.465479   
      H_2850656_ILANGOVAN_V__2850656_19471005_Male_OS...   15.0  0.256684   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   34.0  1.544104   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   29.0  0.942568   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OD.jpg    37.0  0.592740   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OS.jpg    27.0  0.434360   
      H_3489266_SENTHIL KUMAR_R__3489266_19751119_Mal...   29.0  0.350420   
      H_4275323_MEENAKSHI_SUNDARAM__4275323_19520922_...   32.0  0.371829   
      H_429419_SATHYAMOORTHY_NP__429419_19510101_Fema...   36.0  1.009068   
      H_4297748_DILIP_JAIN__4297748_19711011_Male_OS.jpg   45.0  0.729469   
      H_4710617_ANBU HEPZIBAH_MUTHABARANAM__4710617_1...   26.0  0.592575   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   37.0  0.488386   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   33.0  0.307287   
      H_519266_SIB SANKAR_PATRA__519266_19511006_Male...   30.0  0.304308   
      H_902196_VISWANATHAN S___902196_19710803_Male_O...   39.0  0.429858   
M     M_ABHIJIT_ROY__2886965_19990928_Male_OD.jpg          40.0  1.302589   
      M_ABHIJIT_ROY__2886965_19990928_Male_OS.jpg          29.0  0.993074   
      M_ABIHA TAHSIN_CHOWDHURY__2889149_19881204_Fema...   32.0  0.484712   
      M_AMOSE_T__1178927_19920112_Male_OD.jpg              23.0  0.265454   
      M_AMOSE_T__1178927_19920112_Male_OS.jpg              25.0  0.343625   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD.jpg        22.0  0.298417   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD_1.jpg      28.0  0.430584   
      M_Aishwarya_R__1184391_19961122_Female_OD.jpg        20.0  0.406883   
      M_Aishwarya_R__1184391_19961122_Female_OS.jpg        22.0  0.392514   
      M_Akbar_Rukshar__1178658_19940804_Female_OD.jpg      25.0  0.421694   
      M_Akbar_Rukshar__1178658_19940804_Female_OS.jpg 

In [81]:
df.groupby(['Class', 'img_id']).tort.describe()

count      mean  \
Class img_id                                                                
E     E_1193671_Jandyal_sourabh__1193671_19951205_Mal...   30.0  0.013670   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   35.0  0.007413   
      E_1204143_Sureandhiran_R__1204143_19960529_Male...   27.0  0.008476   
      E_1204144_Divya_A__1204144_19940912_Female_OD.jpg    28.0  0.005190   
      E_1204144_Divya_A__1204144_19940912_Female_OS.jpg    24.0  0.007848   
      E_1204147_Malika_M__1204147_19961230_Female_OD.jpg   34.0  0.006161   
      E_1204147_Malika_M__1204147_19961230_Female_OS.jpg   27.0  0.006899   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   28.0  0.004526   
      E_1204148_Jagateeswari_R__1204148_19950809_Fema...   33.0  0.008467   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   35.0  0.010330   
      E_1204149_Muthukumaran_S__1204149_19960423_Male...   33.0  0.004972   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   30.0  0.011450   
      E_1204306_Sangeetha_B__1204306_19970311_Female_...   29.0  0.009584   
      E_209398_SUMIT_CHOUDHARY__209398_19850606_Male_...   27.0  0.009065   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   25.0  0.007201   
      E_258823_Krishna Kumar_Ramani__258823_19701105_...   24.0  0.003949   
      E_301279_Vineet_Ratra__301279_19700517_Male_OD.jpg   30.0  0.010776   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   28.0  0.007319   
      E_595581_GEETHA_PRAKASAM__595581_19520209_Femal...   25.0  0.002195   
      E_787513_SUSANTA_SINGHA__787513_19801019_Male_O...   34.0  0.005830   
H     H_1189801_KOWSIGAN_M__1189801_19950306_Male_OD.jpg   25.0  0.006697   
      H_1189801_KOWSIGAN_M__1189801_19950306_Male_OS.jpg   27.0  0.007179   
      H_2144798_BISWA NATH ADAK_Male_OS.jpg                28.0  0.010423   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   39.0  0.012009   
      H_2502374_HARIHARAN_C S__2502374_19670326_Male_...   35.0  0.004665   
      H_2836849__BALASUBRAMANYAN_S__2836849_19380815_...   36.0  0.009047   
      H_2850656_ILANGOVAN_V__2850656_19471005_Male_OS...   15.0  0.006046   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   34.0  0.006313   
      H_2865623_BIPIN KUMAR_GUPTA__2865623_19511215_M...   29.0  0.015912   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OD.jpg    37.0  0.008761   
      H_2885391_PRAKASH_R__2885391_19780907_Male_OS.jpg    27.0  0.008703   
      H_3489266_SENTHIL KUMAR_R__3489266_19751119_Mal...   29.0  0.006159   
      H_4275323_MEENAKSHI_SUNDARAM__4275323_19520922_...   32.0  0.010728   
      H_429419_SATHYAMOORTHY_NP__429419_19510101_Fema...   36.0  0.009980   
      H_4297748_DILIP_JAIN__4297748_19711011_Male_OS.jpg   45.0  0.005254   
      H_4710617_ANBU HEPZIBAH_MUTHABARANAM__4710617_1...   26.0  0.007425   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   37.0  0.007371   
      H_4725830_ALANGARA JAYARANI_P__4725830_19610505...   33.0  0.004615   
      H_519266_SIB SANKAR_PATRA__519266_19511006_Male...   30.0  0.004242   
      H_902196_VISWANATHAN S___902196_19710803_Male_O...   39.0  0.006339   
M     M_ABHIJIT_ROY__2886965_19990928_Male_OD.jpg          40.0  0.006568   
      M_ABHIJIT_ROY__2886965_19990928_Male_OS.jpg          29.0  0.008777   
      M_ABIHA TAHSIN_CHOWDHURY__2889149_19881204_Fema...   32.0  0.005484   
      M_AMOSE_T__1178927_19920112_Male_OD.jpg              23.0  0.003222   
      M_AMOSE_T__1178927_19920112_Male_OS.jpg              25.0  0.004556   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD.jpg        22.0  0.005654   
      M_AVINASH_KUMAR__2888002_20000306_Male_OD_1.jpg      28.0  0.006504   
      M_Aishwarya_R__1184391_19961122_Female_OD.jpg        20.0  0.004555   
      M_Aishwarya_R__1184391_19961122_Female_OS.jpg        22.0  0.004800   
      M_Akbar_Rukshar__1178658_19940804_Female_OD.jpg      25.0  0.004885   
      M_Akbar_Rukshar__1178658_19940804_Female_OS.jpg 